In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import csv
import pandas as pd
from datetime import date, timedelta
import datetime as dt
import re
import numpy as np

In [2]:
def lastpage(start_date, end_date):
    browser = webdriver.Chrome(executable_path='./chromedriver.exe')
    url = 'https://www.traveloka.com/vi-vn/hotel/search?spec='+start_date+'.'+end_date+'.1.1.HOTEL_GEO.10010083.%C4%90%C3%A0%20N%E1%BA%B5ng.2'
    browser.get(url)
    time.sleep(5)
    browser.find_element_by_css_selector('#last-button').click()
    time.sleep(10)
    html_text = browser.page_source
    browser.quit()
    tree = BeautifulSoup(html_text, 'html.parser')
    pagelast = tree.find('div', {'class': 'lKjqV _2y37E tvat-activeButton'}).string
    return pagelast

In [3]:
def lastpagefac(start_date, end_date, facility):
    browser = webdriver.Chrome(executable_path='./chromedriver.exe')
    url = 'https://www.traveloka.com/vi-vn/hotel/search?spec='+start_date+'.'+end_date+'.1.1.HOTEL_GEO.10010083.%C4%90%C3%A0%20N%E1%BA%B5ng.2'
    browser.get(url)
    time.sleep(50)
    facs = browser.find_elements_by_css_selector('[class = "_1iwut _1bm1f"]')
    for fac in facs:
        if facility in fac.text:
            fac.location_once_scrolled_into_view
            fac.click()
            break
            
    time.sleep(5)
    f = browser.find_element_by_css_selector('#last-button')
    f.location_once_scrolled_into_view
    f.click()
    time.sleep(5)
    html_text = browser.page_source
    browser.quit()
    tree = BeautifulSoup(html_text, 'html.parser')
    pagelast = tree.find('div', {'class': 'lKjqV _2y37E tvat-activeButton'}).string
    return pagelast

In [4]:
#lay ngay (dang xet 1 dem nen phai 2 ngay ke tiep nhau).
today = date.today()
tomorrow = today + timedelta(1)

start_date = dt.datetime.strftime(today, "%d-%m-%Y")
end_date = dt.datetime.strftime(tomorrow, "%d-%m-%Y")

In [ ]:
#Thu thap cac du lieu chinh cua khach san
nextpage = True
pagelast = lastpage(start_date, end_date)
lent = 1
browser = webdriver.Chrome(executable_path='./chromedriver.exe')
browser.get('https://www.traveloka.com/vi-vn/hotel/search?spec='+start_date+'.'+end_date+'.1.1.HOTEL_GEO.10010083.%C4%90%C3%A0%20N%E1%BA%B5ng.2')

names = []
districts = [] 
stars = []
ratings = []
num_ratings = []
prices = []
    

while nextpage:
    time.sleep(5)
    html_text = browser.page_source
    tree = BeautifulSoup(html_text, 'html.parser')
    hotels = tree.find_all('div', {'class': 'ztzlF _2nyWi'})
    

    for hotel in hotels:

        check = hotel.find('div', {'class': '_3ohst Jewfo _2Vswb'}).string
        if 'Khách sạn' not in check:
            continue
            
        name = hotel.find('div', {'class': '_1z5je _10ZQX tvat-hotelName'}).string    
        names.append(name)
        
        district = hotel.find('div', {'class': '_3tICV'}).text.split(',')
        if 'Đà Nẵng' not in district[0]:
            
            districts.append(district[0])
        else:
            districts.append(np.nan)
    
        starf = hotel.find('meta', {'itemprop': 'ratingValue'})
        if starf is None:
            star = np.nan
        else:
            star = starf['content']
        stars.append(float(star))
        
        ratingScore = hotel.find('span', {'class': 'tvat-ratingScore'})
        if ratingScore is None:
            rating = '0'
        else:
            rating = ratingScore.string
        ratings.append(float(rating))
    
        reviews = hotel.find('span', {'class': '_30os4'})
        if reviews is None:
            num_rating = '0'
        else:
            num_rating = reviews.string.strip('()')
        num_ratings.append(int(num_rating))
    
        price = hotel.find('div', {'class': '_22n9I tvat-primaryPrice'}).string.split()
        prices.append(price[0])
    
    
    if lent != int(pagelast):
        time.sleep(5)
        browser.find_element_by_css_selector('#next-button').click()
        nextpage = True
    else:
        nextpage = False
    lent = lent + 1

hotels_df = pd.DataFrame([names, districts, stars, ratings, num_ratings, prices]).T
hotels_df.columns = ['Name', 'District', 'Star', 'RatingScore', 'Num_ratings', 'Price(VND)']
browser.quit()

In [41]:
hotels_df

,Name,District,Star,RatingScore,Num_ratings,Price(VND)
0,Angel Hotel Danang,Sơn Trà,3,8.6,287,162.790
1,Stella Maris Beach Danang,Sơn Trà,4,8.9,63,832.500
2,Khách sạn DLG Đà Nẵng,Sơn Trà,4.5,9.3,62,1.026.908
3,San Marino Boutique Danang,Sơn Trà,4,9,212,600.000
4,Khách sạn Raon Danang Beach,Ngũ Hành Sơn,3,8.8,613,337.920
...,...,...,...,...,...,...
539,Nhat Minh Hotel and Apartment,Ngũ Hành Sơn,3.5,8.2,26,678.805
540,Khách sạn Danang Riverside,Sơn Trà,4,8.4,82,1.760.000
541,Khách sạn Night Sky,Quận Hải Châu,2,8.8,111,414.000
542,Ngoi Nha Xanh Hotel,Quận Hải Châu,2,7.4,146,240.000


In [21]:
#Ham thu thap cac khach san co tien nghi(Ho boi, Cho dau xe,...)
def collect_hotels_facilities(start_date, end_date, facility, name_fac, sleep_time=1):
    nextpage = True
    pagelastfac = lastpagefac(start_date, end_date, facility)
    lent = 1
    browser = webdriver.Chrome(executable_path='./chromedriver.exe')
    browser.get('https://www.traveloka.com/vi-vn/hotel/search?spec='+start_date+'.'+end_date+'.1.1.HOTEL_GEO.10010083.%C4%90%C3%A0%20N%E1%BA%B5ng.2')

    time.sleep(50)
    facs = browser.find_elements_by_css_selector('[class = "_1iwut _1bm1f"]')
    for fac in facs:
        if facility in fac.text:
            fac.location_once_scrolled_into_view
            fac.click()
            break

    names = []
    names_fac = []

    while nextpage:
        time.sleep(5)
        html_text = browser.page_source
        tree = BeautifulSoup(html_text, 'html.parser')
        hotels = tree.find_all('div', {'class': 'ztzlF _2nyWi'})


        for hotel in hotels:

            check = hotel.find('div', {'class': '_3ohst Jewfo _2Vswb'}).string
            if 'Khách sạn' not in check:
                continue

            name = hotel.find('div', {'class': '_1z5je _10ZQX tvat-hotelName'}).string    
            names.append(name)
            
            names_fac.append('Yes')

            

        if lent != int(pagelastfac):
            time.sleep(10)
            f = browser.find_element_by_css_selector('#next-button')
            f.location_once_scrolled_into_view
            f.click()
            nextpage = True
        else:
            nextpage = False
        lent = lent + 1

    df = pd.DataFrame([names, names_fac]).T
    df.columns = ['Name', name_fac]      
    browser.quit()
    return df
    

In [22]:
hotels_pool_df = collect_hotels_facilities(start_date, end_date, 'Hồ bơi', 'Pool')

In [23]:
hotels_pool_df

,Name,Pool
0,Khách sạn Samdi Đà Nẵng,Yes
1,Central Hotel and Spa Danang,Yes
2,Konoha Boutique Villa,Yes
3,Minh Boutique,Yes
4,The Code Hotel & Spa,Yes
...,...,...
100,Golden Sea 3 Hotel,Yes
101,Khách sạn BlueSun,Yes
102,Khách sạn Brilliant,Yes
103,King's Finger Hotel,Yes


In [24]:
hotels_parking_df = collect_hotels_facilities(start_date, end_date, 'Chỗ đậu xe', 'Parking')

In [25]:
hotels_parking_df

,Name,Parking
0,Angel Hotel Danang,Yes
1,Stella Maris Beach Danang,Yes
2,Khách sạn DLG Đà Nẵng,Yes
3,San Marino Boutique Danang,Yes
4,Vinpearl Condotel Riverfront Da Nang,Yes
...,...,...
280,Nhat Minh Hotel and Apartment,Yes
281,Khách sạn Danang Riverside,Yes
282,Khách sạn Night Sky,Yes
283,Ngoi Nha Xanh Hotel,Yes


In [31]:
hotels_restaurant_df =collect_hotels_facilities(start_date, end_date, 'Nhà hàng', 'Restaurant')

In [32]:
hotels_restaurant_df

,Name,Restaurant
0,Angel Hotel Danang,Yes
1,Stella Maris Beach Danang,Yes
2,Khách sạn DLG Đà Nẵng,Yes
3,San Marino Boutique Danang,Yes
4,Khách sạn Raon Danang Beach,Yes
...,...,...
369,Nhat Minh Hotel and Apartment,Yes
370,Khách sạn Danang Riverside,Yes
371,Khách sạn Night Sky,Yes
372,Ngoi Nha Xanh Hotel,Yes


In [33]:
data_hotels_df = pd.merge(hotels_df,hotels_pool_df, how='left' ,on = 'Name')
data_hotels_df = pd.merge(data_hotels_df, hotels_parking_df, how='left', on = 'Name')
data_hotels_df = pd.merge(data_hotels_df, hotels_restaurant_df, how='left', on = 'Name')
data_hotels_df = data_hotels_df.drop_duplicates()

In [34]:
ppe = ['Pool', 'Parking', 'Restaurant']
data_hotels_df[ppe] = data_hotels_df[ppe].fillna('No')
data_hotels_df 

,Name,District,Star,RatingScore,Num_ratings,Price(VND),Pool,Parking,Restaurant
0,Angel Hotel Danang,Sơn Trà,3,8.6,287,162.790,No,Yes,Yes
1,Stella Maris Beach Danang,Sơn Trà,4,8.9,63,832.500,No,Yes,Yes
2,Khách sạn DLG Đà Nẵng,Sơn Trà,4.5,9.3,62,1.026.908,No,Yes,Yes
3,San Marino Boutique Danang,Sơn Trà,4,9,212,600.000,No,Yes,Yes
4,Khách sạn Raon Danang Beach,Ngũ Hành Sơn,3,8.8,613,337.920,No,No,Yes
...,...,...,...,...,...,...,...,...,...
545,Nhat Minh Hotel and Apartment,Ngũ Hành Sơn,3.5,8.2,26,678.805,No,Yes,Yes
546,Khách sạn Danang Riverside,Sơn Trà,4,8.4,82,1.760.000,No,Yes,Yes
547,Khách sạn Night Sky,Quận Hải Châu,2,8.8,111,414.000,No,Yes,Yes
548,Ngoi Nha Xanh Hotel,Quận Hải Châu,2,7.4,146,240.000,No,Yes,Yes


In [35]:
def fix(s):
    if type(s) is not str:
        s = str(s)
    split = re.split('\.', s)
    price = 0
    for i in split:
        price = price*1000 + int(i)
    return price

In [36]:
data_hotels_df ['Price(VND)'] = data_hotels_df ['Price(VND)'].apply(fix)
data_hotels_df 

,Name,District,Star,RatingScore,Num_ratings,Price(VND),Pool,Parking,Restaurant
0,Angel Hotel Danang,Sơn Trà,3,8.6,287,162790,No,Yes,Yes
1,Stella Maris Beach Danang,Sơn Trà,4,8.9,63,832500,No,Yes,Yes
2,Khách sạn DLG Đà Nẵng,Sơn Trà,4.5,9.3,62,1026908,No,Yes,Yes
3,San Marino Boutique Danang,Sơn Trà,4,9,212,600000,No,Yes,Yes
4,Khách sạn Raon Danang Beach,Ngũ Hành Sơn,3,8.8,613,337920,No,No,Yes
...,...,...,...,...,...,...,...,...,...
545,Nhat Minh Hotel and Apartment,Ngũ Hành Sơn,3.5,8.2,26,678805,No,Yes,Yes
546,Khách sạn Danang Riverside,Sơn Trà,4,8.4,82,1760000,No,Yes,Yes
547,Khách sạn Night Sky,Quận Hải Châu,2,8.8,111,414000,No,Yes,Yes
548,Ngoi Nha Xanh Hotel,Quận Hải Châu,2,7.4,146,240000,No,Yes,Yes


In [37]:
data_hotels_df.to_csv('data_hotels.csv', index=0)

In [38]:
test = pd.read_csv('data_hotels.csv')
test

,Name,District,Star,RatingScore,Num_ratings,Price(VND),Pool,Parking,Restaurant
0,Angel Hotel Danang,Sơn Trà,3.0,8.6,287,162790,No,Yes,Yes
1,Stella Maris Beach Danang,Sơn Trà,4.0,8.9,63,832500,No,Yes,Yes
2,Khách sạn DLG Đà Nẵng,Sơn Trà,4.5,9.3,62,1026908,No,Yes,Yes
3,San Marino Boutique Danang,Sơn Trà,4.0,9.0,212,600000,No,Yes,Yes
4,Khách sạn Raon Danang Beach,Ngũ Hành Sơn,3.0,8.8,613,337920,No,No,Yes
...,...,...,...,...,...,...,...,...,...
484,Nhat Minh Hotel and Apartment,Ngũ Hành Sơn,3.5,8.2,26,678805,No,Yes,Yes
485,Khách sạn Danang Riverside,Sơn Trà,4.0,8.4,82,1760000,No,Yes,Yes
486,Khách sạn Night Sky,Quận Hải Châu,2.0,8.8,111,414000,No,Yes,Yes
487,Ngoi Nha Xanh Hotel,Quận Hải Châu,2.0,7.4,146,240000,No,Yes,Yes


In [39]:
test.dtypes

Name            object
District        object
Star           float64
RatingScore    float64
Num_ratings      int64
Price(VND)       int64
Pool            object
Parking         object
Restaurant      object
dtype: object